authors:
- Christophe Gauffre (linkedin: /christophegauffre)
- Aurélien Nanette( linkedin: /a-nanette)

# Build du projet

## Installation de base

In [20]:
from google.colab import drive
import os
import pathlib
import time

drive.mount("/content/gdrive", force_remount=False)

! pip install -q kaggle
! pip install -q plotly --upgrade
! pip install -q streamlit-wordcloud
! pip install -qq streamlit
! pip install -qq pyngrok

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## chargement des données et préparation

In [21]:
t0 = time.time()
#si les données ne sont pas sur le drive, les télécharger de Kaggle. Fichier d'autentif dans le drive
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle" # dossier drive où a été stocké préalablement le fichier json kaggle de credentials
data_dir = pathlib.Path('dataset')

# téléchargement des données
! kaggle datasets download -d daisukelab/dc2020task2 #--force # force pour force le téléchargement
# unzip des données
! unzip -q dc2020task2.zip -d dataset # unzip dans le dossier 'dataset' de la VM google

t1 = time.time()
print("telechargement et unzip effectuée en {:d}mn et {:d}s".format(int((t1-t0)/60),int((t1-t0)%60)))


dc2020task2.zip: Skipping, found more recently modified local copy (use --force to force download)
replace dataset/ToyCar/test/anomaly_id_01_00000000.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
telechargement et unzip effectuée en 0mn et 20s


## Script main.py

In [22]:
%%writefile streamlit_toolbox.py
import pandas as pd
import tensorflow as tf
import os
import streamlit as st
import time

def get_anomaly(file_path):
  """
  dans le cadre du dataset https://www.kaggle.com/daisukelab/dc2020task2
  retourne la classe d'anomalie du fichier son en fonction du nom du fichier
  :param file_path: chemin vers le fichier son étudié
  :return: int: label -  0 / 1: sans anomalie/avec anomalie
  """
  parts = file_path.split(os.path.sep)
  anomaly = parts[-1].split('_')[0]
  if anomaly == 'anomaly':
      return 'oui'
  else:
      return 'non'

def get_machine_id(file_path):
  """
  dans le cadre du dataset https://www.kaggle.com/daisukelab/dc2020task2
  retourne la machine concernée par le fichier
  :param file_path: chemin vers le fichier son étudié
  :return: str: nom de la machine
  """
  parts = file_path.split(os.path.sep)
  machine_id = parts[1] + '_' + parts[3].split('_')[-2]
  return machine_id

@st.cache
def select_inputfile(data_dir, selected_machines):
  """
  méthode de chargement des fichiers, en fonction des type de machines analysées.

  :param data_dir: directory ou sont enregistrés les fichiers
  :param selected_machines: liste de type de machine à étudier
  :return: une dataframe, ainsi que la liste de fichiers train, liste  de fichiers test, liste des type de machine différents, liste de machines différentes,
  dictionnaire des données type/machine/fichier de train et de test
  """
  df  = pd.DataFrame(columns=["fichier","type","set","anomalie"])

  files_train = []
  files_test = []
  for m in selected_machines:
      mach_files_train = pd.DataFrame(tf.io.gfile.glob(str(data_dir) + '/'+m+'/train/*'),columns=['fichier'])
      mach_files_train['type']=m
      mach_files_train['set']='train'
      df = df.append(mach_files_train,ignore_index=True)

      mach_files_test = pd.DataFrame(tf.io.gfile.glob(str(data_dir) + '/'+m+'/test/*'),columns=['fichier'])
      mach_files_test['type']=m
      mach_files_test['set']='test'
      df = df.append(mach_files_test,ignore_index=True)

      files_train.extend(mach_files_train.fichier)
      files_test.extend(mach_files_test)

  machine_list = df['type'].unique()
  df['machine'] = df.fichier.map(get_machine_id)
  df['anomalie'] = df.fichier.map(get_anomaly)
  machine_id_list = df['machine'].unique()

  machine_train_dict = {}
  machine_test_dict={}

  for m in machine_list:
      machine_train_dict[m]={}
      machine_test_dict[m] = {}
      for id in set(df[df['type']==m].machine):
          machine_train_dict[m][id] = df[(df['type']== m) & (df['machine']==id)].fichier
    
  return df,machine_list,machine_id_list,machine_train_dict,machine_test_dict

import cv2 as cv
import urllib.request
import ssl
import numpy as np
#import toolbox2 as tb2

ssl._create_default_https_context = ssl._create_unverified_context

def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  imageBGR = cv.imdecode(image, cv.IMREAD_COLOR)
  imageRGB = cv.cvtColor(imageBGR , cv.COLOR_BGR2RGB)
  return imageRGB

def resize(image,size):
  return cv.resize(image,size,interpolation = cv.INTER_AREA)

#@st.cache
def init_toolbox(machine_list,machine_id_list,spec_size=(512,256)):
  tb2.machine_list = machine_list
  tb2.machine_id_list = machine_id_list
  tb2.spectro_size=spec_size

# @st.cache
# def transform_data_train(mach, machine_train_dict):
#   return tb2.prepare_data_train(mach, machine_train_dict)

# @st.cache(allow_output_mutation=True,suppress_st_warning=True)
# def createData(files, verbose=1):
#   tb2.spectro_size=(512,256)
#   with st.empty():
#       if verbose:
#         st.spinner("création du pipeline de preprocessing des données ...") 
#       t0 = time.time()
#       dataset = tf.data.Dataset.from_tensor_slices(files)
#       dataset = dataset.map(tb2.get_waveform_and_label_and_machine_id_and_anomaly, num_parallel_calls=tb2.AUTOTUNE)
#       dataset = dataset.map(tb2.get_spectrogram_and_label_and_machine_id_and_anomaly, num_parallel_calls=tb2.AUTOTUNE).batch(len(files)) #.batch(1000)

#       if verbose:
#           st.spinner("création des données...")
#       X, label,machine,anomaly = next(iter(dataset)) #bien faire un batch de la longueur voulue

#       t1 = time.time()
#       if verbose:
#           st.spinner("génération des spectrogrammes de {:d} fichiers en {:.2f}s (temps moyen de {:.3f}ms par fichier)".format(len(files),t1-t0,(t1-t0)/(len(files))*1000))

#   return X,label,machine,anomaly

from keras import backend as K
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU, Flatten
from tensorflow.keras.models import Model, Sequential, load_model
AUTOTUNE = tf.data.AUTOTUNE


def createModel():
    """
    Cette méthode crée le modème d'encodage des spectros
    :return: Sequential model
    """

    model = Sequential()
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', input_shape=[512,256,1]))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu')) # tests à 128
    model.add(Dropout(0.2))
    model.add(Dense(128)) # test à 64 un peu moins performant

    return model

   



@st.cache(allow_output_mutation=True)
def loadModel(file):
  """
    méthode de chargement d'un modèle préentraîné, enregistré sur disque
    :param file: chemin d'accès au fichier  de poids
    :return: modèle préentraîné
  """
  session = K.get_session()
  model = createModel()
  model.load_weights(file)
  return model,session

@st.cache(hash_funcs={tf.keras.models.Sequential: lambda model: model.get_config()})
def encode_train(model,data,session):
  encoded = model.predict(data)
  return encoded

@st.cache(hash_funcs={tf.keras.models.Sequential: lambda model: model.get_config()})
def encode_test(model,data,session):
  encoded = model.predict(data)
  return encoded


@st.cache
def  get_data(file,index=None,cols=None):
 df = pd.read_csv(file,index_col=index,usecols=cols)
 return df

@st.cache
def get_data_LOF():
  return pd.read_csv('/content/gdrive/MyDrive/Datascientest/PROJET/PyGoodVibes/STREAMLIT/final_LOF_results.csv',index_col=0)
@st.cache
def get_data_SVM():
  return pd.read_csv('/content/gdrive/MyDrive/Datascientest/PROJET/PyGoodVibes/STREAMLIT/final_SVM_results.csv',index_col=0)


Overwriting streamlit_toolbox.py


In [29]:
%%writefile app.py
import streamlit_toolbox as tb
import streamlit as st
import tensorflow as tf
from keras import backend as K
import keras
import os
import pathlib
import pandas as pd
import numpy as np
import librosa
import librosa.display

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import seaborn as sns
from sklearn.decomposition import PCA
import time

import streamlit_wordcloud as wordcloud

class my_app():

  def __init__(self):

    # self.px_color = px.colors.qualitative.spectro2 # TODO
    # self.px_color = px.colors.qualitative.G10
    self.px_color = px.colors.qualitative.Set2

    sns.set() # Use seaborn's default style to make attractive graphs
    plt.rcParams['figure.dpi'] = 100 # 
    
    self.data_dir = pathlib.Path('dataset')
    self.available_machines = tf.io.gfile.listdir(str(self.data_dir))
    
    self.root_path = "/content/gdrive/MyDrive/git/pyGoodVibes/"
    self.toycar_test_pred_v2 = pd.read_csv(self.root_path+'data/toycar_prediction_v2.csv')

    self.df = pd.read_csv(self.root_path+'final_encoded_machine_dataset.csv',usecols=['fichier','type','set','machine','anomalie'])
    if 'fichier' not in self.df.columns:
        self.df = self.df.reset_index()
        self.df = self.df.rename(index={0:'fichier'})

    self.machine_list = self.df['type'].unique()
    self.machine_id_list = self.df.machine.unique()

    #tb.init_toolbox(self.machine_list,self.machine_id_list)

    nom_final = self.root_path+'final__weight_encoder_model_machine.h5'
    self.encoder, session = tb.loadModel(nom_final)

    st.image(f"{self.root_path}images/main_banner.png")

    st.sidebar.title("pyGoodVibes\n")
    st.sidebar.title("Menu\n")

    menu = ["Le projet","Les données","Modélisation","Résultats & Perspectives","A propos"]
    self.choice = st.sidebar.selectbox("", menu,index = 0)

    if self.choice == 'Le projet':
      self.chapitre_choice = self.chapter_selection_on_sidebar(["Résumé",
                                                                "Contexte",
                                                                "Progression",
                                                                "Résultats et Perspectives"])
      self.le_projet()

    elif self.choice == 'Les données':
      self.sous_menu_donnees = ["Première exploration de données","Visualisation des données audio","Visualisation temps-fréquence"]
      self.chapitre_choice = self.chapter_selection_on_sidebar(self.sous_menu_donnees)
      self.machine_selection_on_sidebar()
      self.les_donnees()

    elif self.choice == "Modélisation":
      self.modelisation_chapters = ["intro - objectifs",
                                "fonction de perte",
                                "architecture",
                                "résultats de l'encodage",
                                "discrimination des anomalies",
                                "détection non supervisée des anomalies"]
      self.chapitre_choice = self.chapter_selection_on_sidebar(self.modelisation_chapters)
      self.machine_selection_on_sidebar()
      self.modelisation()
    elif self.choice == "Résultats & Perspectives":
      self.machine_selection_on_sidebar()
      self.resultats()
    elif self.choice == "A propos":
      self.a_propos()
    
    elif choice == "Biblio":
      self.biblio()


  def chapter_selection_on_sidebar(self,chapters_list):
      # chapters_list = ["Résumé","Contexte","Progression","Résultats et Perspectives"]
      with st.sidebar:
        chapitre_choice = st.radio("chapitre",chapters_list)
      return chapitre_choice


  def machine_selection_on_sidebar(self):
    with st.sidebar.form(key ='Form1'):
      if 'selected_machines' not in st.session_state:
        st.session_state.selected_machines = [
                                              self.machine_list[0],
                                              self.machine_list[1],
                                              self.machine_list[2],
                                              self.machine_list[3],
                                              self.machine_list[4],
                                              self.machine_list[5],
                                              ]
      self.selected_machine = st.multiselect('Type de machines testées',self.machine_list,key='selected_machines',help='filtre les données affichées') #default=[self.available_machines[i] for i in np.random.choice(range(len(self.available_machines)),np.random.randint(1,len(self.available_machines),1)[0],replace=False)]
      submit_button = st.form_submit_button(label='appliquer', help='applique le nouveau filtre')

    self.update_sample = st.sidebar.button("update",on_click=self.update_samples, help="génére de nouveaux tirages aléatoire de données")
    self.df = self.df[self.df.type.isin(self.selected_machine)]
    self.df_train = self.df[self.df.set=="train"]
    self.df_test = self.df[self.df.set=="test"]

  def le_projet(self):

    if self.chapitre_choice == "Résumé":
      self.projet_resume()

    if self.chapitre_choice == "Contexte":
      self.projet_context()

    if self.chapitre_choice == "Progression":
      self.projet_progression()

    if self.chapitre_choice == "Résultats et Perspectives":
      self.choice == "Résultats & Perspectives"
      self.machine_selection_on_sidebar()
      self.resultats()

 
  def projet_resume(self):
    st.markdown(
        """
        \#AnormalyDetection \#Spectrogramme \#DeepLearning \#Encoder \#ACP \#LOF

        Ce Projet a pour but de **détecter des sons anormaux**, sur des échantillons de 10s enregistrés en condition réelle, à l'aide d'un algorithme d'**apprentissage non supervisé**.

        Plusieurs types de machines ayant des **signatures sonores** très différentes les unes des autres vont constituer notre jeu de données, et comme en condition réelle,
        *seuls les sons normaux vont participer à l'apprentissage du modèle*.

        Lors des phases d'apprentissage et de prédiction (détection d'anormalité), les échantillons sonores sont transformés en **spectrogrammes** pour alimenter le modèle.

        Après plusieurs itérations, notre choix s'est tourné vers l'utilisation d'un modèle de **deep learning** construit sur la base d'un **encoder**, complété d'une **Analyse en Composantes Principales** (ACP),
        pour la partie *feature engineering* (extraction l'information pertinente des spectrogrammes) et d'un **Local Outlier Factor** (LOF) pour la partie *outlier detection*.


        Alors que la différenciation entre sons normaux et anormaux est **quasi imperceptible à l'oreille humaine**, les performances de l'algorithme sont très bonnes sur la moitiés des machines testées!

        """
    )
    st.image(f"{self.root_path}/images/model_v3.png",caption="schéma du pipeline complet")


  def word_cloud_st(self):
    words = [
        dict(text = "sons anormaux", value = 10, commentaire = ""),
        dict(text = "détection précoce", value = 10, commentaire = ""),
        dict(text = "detection automatique", value = 10, commentaire = ""),
        dict(text = "bon marché", value = 10, commentaire=""),
        dict(text = "industrie 4.0", value = 10, commentaire = "L'industrie 4.0 marque le développement d’un système décentralisé, où les processus sont contrôlés et corrigés de façon automatisée et où vous pouvez améliorer les performances des systèmes de production, en tirant parti des nouvelles technologies, et ainsi de les utiliser de manière plus en plus rentable et efficace. Finalement mais pas moins important, l’industrie 4.0 est déterminée à rendre les processus encore plus intelligents, facilitant toujours plus l’utilisation des outils par l’utilisateur."),
        dict(text = "smart factory", value = 10, commentaire = "La Smart Factory – Usine intelligente – est l'objectif de l'industrie 4.0, où la technologie devient un catalyseur pour obtenir une usine interconnectée, plus intelligente et performante grâce à une meilleure collaboration homme-machine."),
        dict(text = "deep learning", value = 10, commentaire = "Le deep learning ou apprentissage profond est un type d'intelligence artificielle dérivé du machine learning (apprentissage automatique) où la machine est capable d'apprendre par elle-même, contrairement à la programmation où elle se contente d'exécuter des règles prédéterminées."),
        dict(text = "IA", value = 10, commentaire = "L'intelligence artificielle (IA) est « l'ensemble des théories et des techniques mises en œuvre en vue de réaliser des machines capables de simuler l'intelligence humaine »"),
        dict(text = "maintenance prédictive", value = 10, commentaire = "La maintenance prédictive consiste à anticiper les défaillances à venir sur un équipement, un objet, un système, etc. Concrètement, il s’agit d’aller au-devant d’une panne ou d’un dysfonctionnement grâce au cumul d'un ensemble de données. En plein essor ces dernières années, la maintenance prédictive permet surtout d’anticiper ces pannes et offre la possibilité d’intervenir en évitant une réparation beaucoup plus coûteuse."),
        dict(text = "apprentissage non-supervisé", value = 10, commentaire = "L'apprentissage non supervisé désigne la situation d'apprentissage automatique où les données ne sont pas étiquetées. Il s'agit donc de découvrir les structures sous-jacentes à ces données non étiquetées. Puisque les données ne sont pas étiquetées, il est impossible à l'algorithme de calculer de façon certaine un score de réussite. L'absence d'étiquetage ou d'annotation caractérise les tâches d'apprentissage non supervisé et les distingue donc des tâches d'apprentissage supervisé."),
        dict(text = "auto-encodeur", value = 10, commentaire = "Les auto-encodeurs sont des algorithmes d’apprentissage non supervisé à base de réseaux de neurones artificiels, qui permettent de construire une nouvelle représentation d’un jeu de données. Généralement, celle-ci est plus compacte, et présente moins de descripteurs, ce qui permet de réduire la dimensionnalité du jeu de données. L’architecture d’un auto-encodeur est constitué de deux parties : l’encodeur et le décodeur."),
        dict(text = "ASD", value = 10, commentaire ="La détection de sons anormaux (Anormalous Sound Detection) est la tâche d'identifier si le son émis par une machine cible est normal ou anormal. La détection automatique des défaillances mécaniques est une technologie essentielle dans la quatrième révolution industrielle, y compris l'automatisation d'usine basée sur l'intelligence artificielle (IA). La détection rapide d'une anomalie de la machine en observant ses sons peut être utile pour la surveillance de l'état de la machine."),
        dict(text = "détection d'anomalies", value = 10, commentaire="Le but de la détection d’anomalie est de repérer des données qui ne sont pas conformes à ce à quoi l’on peut s’attendre par rapport aux autres données. Il s’agit, par exemple, de données qui ne suivent pas le même schéma ou qui sont atypiques pour la distribution de probabilité observée. La difficulté du problème provient du fait qu’on ne connait pas au préalable la distribution sous-jacente de l’ensemble des données. C’est à l’algorithme d’apprendre une métrique appropriée pour détecter les anomalies. Parmi les exemples d’applications courantes, citons les transactions bancaires (où une anomalie sera vue comme une fraude potentielle), la surveillance des données physiologiques d’un malade (l’anomalie est un problème de santé possible), ou encore la détection de défauts dans des chaines de production. La détection d’anomalie est souvent un problème d’apprentissage de type non supervisé."),
        dict(text = "spectrogramme", value = 10, commentaire="Le spectrogramme est un diagramme représentant le spectre d'un phénomène périodique, associant à chaque fréquence une intensité ou une puissance 1. L'échelle des fréquences et celle des puissances ou intensités peuvent être linéaires ou logarithmiques."),
        ]
    return_obj = wordcloud.visualize(words,
                                tooltip_data_fields={"text":"mot-clé", "commentaire":""},
                                per_word_coloring=False,
                                width = "100%",
                                height = 900,
                                layout = 'archimedean',
                                padding = 12,
                                )

  def projet_context(self):
    col1, col2 = st.columns(2)
    with col1:
      self.word_cloud_st()
    with col2:
      expander_context = st.expander("Contexte",expanded=True)
      with expander_context:
          st.markdown("""
        La **détection de sons anormaux** (ASD - Anomalous Sound Detection) a pour but d'identifier si le son émis par une source quelconque, est normal ou anormal,
        alors qu’un son qualifié d’anormal n’est pas connu auparavant.
      
        Cette détection d’anomalie non supervisée est utilisée de manière très intuitive par l’homme, dans tout domaine de la maintenance :
        par exemple dans notre quotidien, si nous entendons un bruit suspect lors de l’utilisation de notre voiture,
        nous la portons au garage pour diagnostic et réparation. Dans l’industrie, les équipes de maintenance font des tournées de contrôle afin de vérifier,
        visuellement mais aussi par le son, que le comportant des machines surveillées est normal.
        
        >*Le son et la détection de ces dérives est donc une première source d’information importante dans le diagnostic de panne.*

        A l’ère de la **smart factory** et de la 4ème révolution industrielle (« l’Usine 4.0 »), la **détection automatique des défaillances** par analyse sonore
        par intelligence artificielle est une technologie essentielle pour automatiser ces contrôles  
        
        L’apparition d’anomalies et d’incidents sur les composants peut avoir des conséquences néfastes importantes pour les exploitants et les mainteneurs telles que la perte de production,
        l’interruption de service, les blessures physiques ou encore l’augmentation des coûts de maintenance liée au remplacement de pièces qui aurait pu être évité.
        Ainsi, la **détection précoce** de ces anomalies de fonctionnement peut éviter des dommages plus importants et réduire les coûts de réparation et d'entretien.
        Une détection précoce et fiable des anomalies permet en outre d’identifier l’état d’une machine ou d’un système qui se dégrade,
        permettant ainsi le développement et l’implantation de **maintenance prédictive**, plutôt qu’une maintenance préventive qui coûte parfois très cher.

        La surveillance acoustique a pour ultime avantage le caractère **bon marché** du matériel, ce qui facilite grandement le **déploiement à grande échelle** de cette méthode.

        Le principal défi de cette tâche est de détecter des sons anormaux, alors qu’ils inconnus.
        En effet, dans les usines du monde réel, les sons anormaux réels se produisent rarement et sont très divers. Par conséquent,
        il est impossible de créer et/ou de collecter délibérément des modèles exhaustifs de sons anormaux. Cela signifie que nous devons **détecter des sons anormaux inconnus** qui n'ont jamais été observés.
        Ce point est l'une des principales différences entre l'ASD pour les équipements industriels et les tâches de classification supervisées,
        qui eux détectent les sons anormaux déjà définis et connus, tels que les coups de feu ou les pleurs d'un bébé.

          >*Ce projet se situe donc bien dans le domaine de la détection d’anomalies non-supervisé.*

        Alors que diverses approches de classification supervisées de scènes acoustiques ont été proposées depuis le milieu des années 2010
        (‘Anomalous Sound Detection with Machine Learning : A Systematic Review’, E. NUNES, 2021), la détection d'anomalies acoustiques est encore sous-représentée.
        En raison de la publication récente d'ensembles de données, la situation s'améliore progressivement.

        Actuellement, la majorité des approches de détection d'anomalies acoustiques basées sur des **réseaux de neurones profonds**
        et ne nécessitant pas de connaissance de la machine cible sont activement étudiées, notamment les architectures basées sur l’**auto-encodeur**.
      
      """)


  def projet_progression(self):
    st.markdown("""
      **Acentuer sur l'objectif et l'evolution des méthodes**
    """)
    expander_v1 = st.expander("V1 : Classification à l'aide de couches denses",expanded=False)
    expander_v2 = st.expander("V2 : Detection d'anomalies à l'aide de l'auto-encodeur",expanded=False)
    expander_v3 = st.expander("V3 : Detection d'anomalies par encodage puis classification",expanded=False)

    with expander_v1:
      st.markdown("""### Objectif :
  Evaluer la **capacité de classification** des différents individus d’un même type de machine, grâce à un réseau constitué de **couches denses**
    
        Optimiseur : Adam
        Loss       : sparse_categorical_crossentropy
        Métrique   : Accuracy = 79% sur les machines ToyCar""")

      st.image(f"{self.root_path}/images/schema_v1.png",caption="Modèle et résultats de la première itération du projet")
      st.markdown("""
        ### Bilan :
        Une structure simple composée de couches dense permet déjà de séparer les machines sur la base de spectrogrammes
      """)
    with expander_v2:
      st.markdown("""
      ### Objectif :
        1. Evaluer la capacité de l'autoencodeur pour **capturer l'essentiel de l'information** d'un spectrogramme.
        2. Evaluer si l'**erreur de reconstruction** du spectrogramme est une métrique prometteuse pour dtecter les sons anormaux

      ### Hypothèse : 
      Etant donné que l'apprentissage est effectué uniquement sur le jeu de donnée 'normal', les sons anormaux pourraient présenter une erreur de reconstruction 
      plus élevée (MSE globale entre l'image d'input et l'image d'output du décoder) que les sons normaux.
          
              Optimiseur : Adamax
              Loss       : MSE 
              Métrique   : RMSE significativement plus importante pour les anormaux (pour 4/8 machines testées)""")
      
      # affichage modèle v2
      st.image(f"{self.root_path}/images/model_v2.png",caption="Modèle de la deuxième itération du projet (autoencoder)")

      # affichage tableau et graphique
      col1, col2 = st.columns(2)
      
      with col1:
        st.write("")
        st.write("")
        st.markdown("""Resultats des métriques mse/rmse de 10 échantillons pour le type de machine _Toycar_""")
        st.write("")
        st.write(self.toycar_test_pred_v2[["type_set","classe","type_machine", "machine_type_id","mse","rmse"]].sample(10))

      with col2:
        anorm_1 = self.toycar_test_pred_v2[(self.toycar_test_pred_v2["machine_type_id"]==1)&(self.toycar_test_pred_v2["classe"]=="anomaly")]["rmse"]
        anorm_2 = self.toycar_test_pred_v2[(self.toycar_test_pred_v2["machine_type_id"]==2)&(self.toycar_test_pred_v2["classe"]=="anomaly")]["rmse"]
        anorm_3 = self.toycar_test_pred_v2[(self.toycar_test_pred_v2["machine_type_id"]==3)&(self.toycar_test_pred_v2["classe"]=="anomaly")]["rmse"]
        anorm_4 = self.toycar_test_pred_v2[(self.toycar_test_pred_v2["machine_type_id"]==4)&(self.toycar_test_pred_v2["classe"]=="anomaly")]["rmse"]

        norm_1 = self.toycar_test_pred_v2[(self.toycar_test_pred_v2["machine_type_id"]==1)&(self.toycar_test_pred_v2["classe"]=="normal")]["rmse"]
        norm_2 = self.toycar_test_pred_v2[(self.toycar_test_pred_v2["machine_type_id"]==2)&(self.toycar_test_pred_v2["classe"]=="normal")]["rmse"]
        norm_3 = self.toycar_test_pred_v2[(self.toycar_test_pred_v2["machine_type_id"]==3)&(self.toycar_test_pred_v2["classe"]=="normal")]["rmse"]
        norm_4 = self.toycar_test_pred_v2[(self.toycar_test_pred_v2["machine_type_id"]==4)&(self.toycar_test_pred_v2["classe"]=="normal")]["rmse"]

        hist_data = [anorm_4, norm_4, anorm_3, norm_3, anorm_2, norm_2, anorm_1, norm_1 ]
        hist_label = ["machine_4  -Anormale", "machine_4  Normale",
                      "machine_3  -Anormale", "machine_3  Normale",
                      "machine_2  -Anormale", "machine_2  Normale",
                      "machine_1  -Anormale", "machine_1  Normale"]
        colors = ['rgb(400, 0, 200)','rgb(0, 0, 200)', 'rgb(400, 70, 200)','rgb(0, 70, 200)','rgb(400, 160, 200)','rgb(0, 160, 200)','rgb(400, 220, 200)','rgb(0, 220, 200)']
        fig = ff.create_distplot(hist_data,hist_label,show_hist=False,bin_size=0, colors=colors)
        fig.update_layout(title_text='Densité des RMSE par machine et type de normalité (sous ensemble Toycar)')
        fig.update_xaxes(title_text='RMSE')
        fig.update_xaxes(range=[0.04, 0.19])
        # fig.update_yaxes(title_text='densité')
        st.plotly_chart(fig)

      st.markdown("""
        ### Bilan :
        **L'erreur de reconstruction de l'autoencoder** permet de séparer les sons normaux des sons anormaux.
        Cependant un **traitement plus fin ou plus spécifique** semble nécessaire pour certaines machines, dont les sons normaux/anormaux sont encore indiscernables

      """)



    with expander_v3:
      st.markdown("""### Objectif :
  1. Affiner les caractérisation (individualisation) des machines et des individus   *  --> fonction de perte personnalisée*
  2. Optimiser la détection des anomalies par type machine   *  --> ACP puis LOF*

### Hypothèse :
  La partie *encoder* va être utilisée pour capturer la spécificité des machines.

  1. Grâce à une **fonction de perte personnalisée** :
    - forcer l'identification des **points communs** entre deux spectrogrammes d'une **machine identique**,
    - forcer l'identification des **différences** sur les spectrogrammes de 2 machines **différentes**


  2. Grâce à une **ACP** suivie d'un **LOF** :
    - **Capturer l'essentiel** de l'information dans un **espace réduit** _(à 3 dimensions)_ pour permettre...
    - **L'optimisation par type de machine** de la détection d'anomalie _(en fonction de la dispersion des différents nuages de points)_


    Optimiseur : Adamax
    Loss       : Personnalisée grâce à un générateur de batch de 'triplets de spectrogrammes'
    Métrique   : Sensibilité et spécificité de détection
    """)
      

      st.image(f"{self.root_path}/images/model_v3.png",caption="schéma de la troisième itération du projet : encodage puis classification")
      st.markdown("""
      ### Bilan : [_i_](https://fr.wikipedia.org/wiki/Sensibilit%C3%A9_et_sp%C3%A9cificit%C3%A9#/media/Fichier:Sensibilit%C3%A9_et_Sp%C3%A9cificit%C3%A9.svg)
      - ** Sensibilité volatile** selon les _types de machine_ : 
        - _ToyConveyor_ : 14% de détection _vs_ _Valve_ : 96% de détection !
        - en moyenne autours de **75% de detection d'anomalies**.


      - **Bonne spécificité** de la détection d'anomalie : **15% de fausses alarmes** seulement !

    

      """)

      

  def projet_resultats(self):
    st.markdown("""
      Pointer vers le menu **Résultats & Perspectives**
    """)

  def les_donnees(self):
    st.write("Les données issues du dataset [KAGGLE](https://www.kaggle.com/daisukelab/dc2020task2) sont constituées de fichiers au format _wav_." 
    "\nDiverses manipulations de ces données permettent de mieux cerner où sera le travail de modélisation")
     
    #sous_menu = ["Première exploration de données","Visualisation des données audio","Visualisation temps-fréquence"]
    #ss_menu = st.selectbox("",sous_menu )
    st.header(self.chapitre_choice)

    if self.chapitre_choice==self.sous_menu_donnees[1]:
      self.raw_data()
    elif self.chapitre_choice==self.sous_menu_donnees[0]:
      self.distribution_data()
    elif self.chapitre_choice == self.sous_menu_donnees[2]:
      self.pipeline()

  def pipeline(self):
    #st.subheader("Représentation en temps - fréquence à l'aide de spectrogramme")
    st.write(
    '''
    La **représentation temps-fréquence consiste à extraire les composantes fréquentielles du signal au cours du temps**.
     Les transformations de **fourier** sous-jacentes travaillent sur des fenêtres temporelles du signal, permettant d'analyser chaque sous fenêtre. 
     Si des différences fréquentielles au cours du temps ont lieu dans le signal, elles apparaissent alors dans le **spectrogramme**, représentation matricielle des composantes fréquentielles en colonne, au cours du temps: les lignes 

    ''')

    for m in self.selected_machine:
      if 'expand_'+m not in st.session_state:
        st.session_state['expand_'+m] = False

      if 'sample_normal_'+m not in st.session_state:
          st.session_state['sample_normal_'+m]  = self.df[(self.df['type']==m) & (self.df["anomalie"]=='non')].sample(1).fichier.values[0]
      file1 = st.session_state['sample_normal_'+m]

      if 'sample_anormal_'+m not in st.session_state:
          st.session_state['sample_anormal_'+m] = st.session_state['sample_anormal_'+m] = self.df[(self.df['type']==m) & (self.df["anomalie"]=='oui')].sample(1).fichier.values[0]
      file2 = st.session_state['sample_anormal_'+m]

      my_expander = st.expander(label='visualisation signaux machine '+ m, expanded = st.session_state['expand_'+m])
      
      with my_expander:
        fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(nrows=2, ncols=2, sharex=True,figsize=(18,6),gridspec_kw={'height_ratios': [1, 2]})
        self.plot_audio(file1,fig,ax1)
        ax1.set(title='signal '+m+' normal')
      
        self.plot_audio(file2,fig,ax2)
        ax2.set(title='signal '+m+' anormal')

        img,spectro1=self.plot_spectrogram(file1,ax3)
        ax3.set(title='spectrogramme')
       
        _,spectro2=self.plot_spectrogram(file2,ax4)
        ax4.set(title='spectrogramme')

        cbar_ax = fig.add_axes([0.135, 0.00, 0.75, 0.02]) #0.09, 0.06, 0.84, 0.02
        fig.colorbar(img, cax=cbar_ax, orientation="horizontal")
        st.write(fig)
    
  def distribution_data(self):
    st.write('''
    Les fichiers du dataset sont fournis dans différents répertoires, classés par _**type de machine**_, et jeu d'_**apprentissage**_ ou de _**test**_. Une dataframe dans laquelle les données ont été _dénormalisées_ a été constituée pour faciliter la manipulation. 
    Le dataset est constitué de près de 31 000 enregistrements pour 6 types de machines: 86h d'enregistrement et près de 10 Gb de données.  Chaque type de machine comporte 3 ou 4 machines différentes. 
    ''')
      
    if 'check_data' not in st.session_state:
      st.session_state.check_data = False
    check_data = st.checkbox("afficher toutes les données",key='check_data',help='affiche seulement 20 lignes aléatoires de la dataframe, ou la totalité des lignes')

    if 'df_samples' not in st.session_state:
      st.session_state.df_samples = np.random.choice(len(self.df),20,False)

    if check_data:
      st.write(self.df)
    else: 
      valid_index = [i for i in st.session_state.df_samples if i<len(self.df) ]
      st.write(self.df.iloc[valid_index].sort_values('type'))

    st.subheader("répartition des fichiers et de leurs caractéristiques")
    
    abs = st.checkbox("affichage absolu",help="cliquez sur les figures pour afficher les détails")
    lab_type= 'label+percent entry'
    if abs:
      lab_type='label+value'

    col1,col2 = st.columns(2)
    color_sequence = self.px_color
    color_map={}
    color_map['set']='#FFFFFF'
    for i, m in enumerate(self.df.type.unique()):
      color_map[m]=color_sequence[i]
    

    self.df_train.sort_values('machine')
    self.df_test.sort_values('machine')
    fig2 = px.sunburst(self.df_train,path=['set','type','machine','anomalie'], color = 'type',color_discrete_map=color_map, title="données d'apprentissage")
    fig2.update_traces(textinfo=lab_type)
    fig2.update_layout(height=800,uniformtext=dict(minsize=15,mode='hide'),margin = dict(t=50, l=0, r=50, b=0), title_font_size=24)
    
    col1.plotly_chart(fig2,use_container_width=True)
  
    fig3 = px.sunburst(self.df_test,path=['set','type', 'machine','anomalie'], color = 'type',color_discrete_map=color_map,title="répartition des données de test")
    fig3.update_layout(height=800,uniformtext=dict(minsize=15,mode='hide'),margin = dict(t=50, l=50, r=0, b=0),title_font_size=24)
    fig3.update_traces(textinfo=lab_type)
    col2.plotly_chart(fig3,use_container_width=True)


  def raw_data(self):

    st.write('''
    Les fichiers sont enregistrés au **format _wav_**, de durée d'environ **10s**. La fréquence d'échantillonnage est identique pour tous les fichiers:  **16kHz**.
    \nCette page permet, à gauche, d'**écouter et de visualiser** des sons pour laquelle la machine ne présente pas d'anomalie, et à droite, des sons présentant une anomalie.
    ''')
    st.caption("Cliquez sur la machine que vous souhaitez étudier pour afficher les informations correspondantes. Vous pouvez sélectionner des machines supplémentaires sur le menu de droite.")

    for m in self.selected_machine:
      if 'expand_'+m not in st.session_state:
        st.session_state['expand_'+m] = False

      my_expander = st.expander(label='visualisation signaux machine '+ m, expanded = st.session_state['expand_'+m])

      with my_expander:
        col1,col2 = st.columns(2)
        col1.write("son normal")
     
        if 'sample_normal_'+m not in st.session_state:
          st.session_state['sample_normal_'+m]  = self.df[(self.df['type']==m) & (self.df["anomalie"]=='non')].sample(1).fichier.values[0]
        file1 = st.session_state['sample_normal_'+m]

        col1.caption(file1)
        col1.audio(open(file1, 'rb').read(),format='audio/wav')

        col2.write("son anormal")

        if 'sample_anormal_'+m not in st.session_state:
          st.session_state['sample_anormal_'+m] = st.session_state['sample_anormal_'+m] = self.df[(self.df['type']==m) & (self.df["anomalie"]=='oui')].sample(1).fichier.values[0]
        file2 = st.session_state['sample_anormal_'+m]

        col2.caption(file2)
        col2.audio(open(file2, 'rb').read(),format='audio/wav')

        fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True,figsize=(18,3))
        ax[0].set(title=m+' normal')
        ax[0].set_ylim(-0.45,0.45)
        self.plot_audio(file1,fig,ax[0])
        ax[1].set(title=m+' anormal')
        ax[1].set_ylim(-0.45,0.45)
        img = self.plot_audio(file2,fig,ax[1])
        st.write(fig)

  def modelisation(self):
    st.header("Modélisation")

    if self.chapitre_choice==self.modelisation_chapters[0]:
      self.model_intro()
    elif self.chapitre_choice==self.modelisation_chapters[1]:
      self.model_loss()
    elif self.chapitre_choice==self.modelisation_chapters[2]:
      self.model_architecture()
    elif self.chapitre_choice==self.modelisation_chapters[3]:
      self.model_scatter()
    elif self.chapitre_choice==self.modelisation_chapters[4]:
      self.model_anomaly()
    else:
      self.model_detection()

  def model_intro(self):

    st.subheader("principes & objectifs:")
    st.markdown(
    """
    La modélisation va consister à **analyser les spectrogrammes**. Nous nous sommes posé plusieurs types de questions pour converger au cours de **différentes itérations** vers l'objectif du projet en lui-même:
      - permettent-ils de **discriminer** chaque type de **machine** ?
      - permettent-ils de **discriminer chaque machine différente**, à l'intérieur de leur propre groupe et parmi tous les types? 
      - peut-on **distinguer** (et donc classifier) **les sons normaux et anormaux**?
      - et enfin, cette classification peut-elle être menée sous la **forme non supervisée**? 
    """
    )
    st.subheader("processus mis en place:")

    st.write('''
    On commence par **encoder les spectrogrammes** (_image embedding_), en recherchant 2 objectifs:
    - réduire leur **taille**
    - **construire un espace discriminant**: l'espace des features construit doit maximiser l'écart entre les différentes machines, et minimiser l'écart entre les différents échantillons d'une même machine, afin de constituer des **grappes homogènes**.
    Nous visons à ce que les **anomalies se détachent de ces grappes**, afin de mettre en oeuvre des **techniques de détection d'anomalies et d'outliers**. 
    
    L'apprentissage est réalisé grâce à un **générateur**, qui produit aléatoirement des batchs de triplets de 2 spectrogrammes d'une même machine et 1 spectrogramme d'une autre machine.
    Il est réalisé sur la totalité du dataset d'entraînement, en moins de 3h (batch: 12 triplets de spectrogrammes, 40 epochs, nb steps: nb_moyens d'images par machine/ taille du batch)
    
    Cette première étape permet de passer d'une **matrice de 512*256** flottants à un **vecteur de 128** flottants.
    
    Afin de réduire encore la complexité du problème de détection, on ajoute après cette représentation continue (_embedding_), une phase **d'Analyse en Composantes Principales**,
     afin de projeter nos nouvelles données créées dans un nouvel espace où les dimensions sont les plus décorrélées possibles. 

    Notre process complet permet donc de passer d'un son de 10s, représenté par **160 000 points** (10s * 16kHz) **à une représentation en 3 dimensions** seulement, permettant de les discriminer visuellement.
    Le dataset de 8 Go compressé est représenté par un fichier csv de 3Mo ! 
    Cet encodage se réalise en 10ms en moyenne sur de Google engine GPU, et 1,5 ms sur les TPU de Google, grâce à un pipeline TensorFlow:  cela permet largement d'envisager une analyse en temps réel.
    ''')

  def en_travaux(self):
    st.subheader("en construction ! ")
    st.markdown(":construction:")

  def model_architecture(self):
    st.write('''
    Pour minimiser la **fonction de _loss_** précédemment décrite, nous avons besoin de calculer, 
    à chaque itération de l'apprentissage, les distances 2 à 2 des vecteurs de sortie d'un **triplet de spectrogrammes**, 
    constitué par 2 spectrogrammes de la même famille, et un 3ème spectrogramme d'une autre famille.

    Le **réseau convolutif** est par conséquent alimenté par **batch de triplets de spectrogrammes**. Ces batchs sont construits grâce à un **générateur**.
  
    ''')
  #st.write(encoder.summary())
    dot_img_file = '/tmp/model_1.png'
    tf.keras.utils.plot_model(self.encoder, to_file=dot_img_file, show_shapes=True,show_dtype=False,show_layer_names=False,expand_nested=False,dpi=60,rankdir = 'TB') #'TB'

    col1,col2 = st.columns((1,2))
    col1.subheader("structure du modèle de CNN")
    col1.subheader("")
    col1.image(dot_img_file)
    col2.subheader("générateur")
    col2.write("")
    file = self.root_path+"images/spectro-triplets.png"
    col2.image(file,use_column_width=True)

    st.subheader("Apprentissage")
    st.write('''
    L'apprentissage est effectué en 2 temps, en faisant varier le learning rate de $5.10^-4$ à $10^-5$.
    Pour ces 2 étapes d'apprentissage, 40 epochs de 675 batch, soit 27 000 triplets de spectrogrammes parmi $20 000^3$ possibles, sont utilisés à chaque apprentissage.

    Il est effectué approximativement en 3h sur un notebook colab pro, disposant de GPU et de 25 Go de RAM. Rappelons que le réseau est très gros: 4 000 000 de paramètres à entraîner.
    
    ''')
    st.caption('NB: Des améliorations dans le processus de calcul des spectrogrammes serait à apporter pour alléger ce recours à la RAM.')

    col1,col2 = st.columns(2)
    col1.image(self.root_path+"images/apprentissage_phase1.PNG")
    col2.image(self.root_path+"images/apprentissage_phase2.PNG")


  def model_loss(self):
    st.subheader("fonction de perte personnalisée: ")
    # $\sum_{\mathclap{1\le i\le 3}} \Delta_i$
    st.write("")
    col1,col2,col3 = st.columns((1,1,1))
    #col1,col2 = st.columns(2)
    file = self.root_path+"images/kmeans.png"
    col1.image(file)
    col2.subheader("$\displaystyle LOSS=\dfrac{\delta_1}{\dfrac{1}{2}(\delta_2 + \delta_3 +\epsilon) }$") #
    file = self.root_path + "images/distances2.png"
    #col2.image(file)
    col3.image(file)

    with st.echo():
      def loss(y_true, y_pred):
        """
        méthode de caclcul de perte cherchant à minimiser la distance entre les 2 premiers vecteurs,
        tout en maximisant leur distance avec le 3eme.

        :param y_true: vecteur théorique attndu: non pris en compte
        :param y_pred: vecteur calculé
        :return: distance moyenne entre les vecteurs du batch
        """
        p1_id1 = y_pred[::3]
        p1_id2 = y_pred[1::3]
        p2_id1 = y_pred[2::3]
        return tf.reduce_mean(normL2(p1_id1, p1_id2)/(1e-8+1/2* normL2(p1_id1, p2_id1) + 1/2* normL2(p1_id2, p2_id1)))

    st.write('''
    Ce **coût** est construit comme fonction des **distances euclidiennes** $\delta_1$, $\delta_2$ et $\delta_3$ entre 3 vecteurs d'un triplet de points $(p_1,p_2,p_3)$. 

    Il est totalement indépendant d'une variable cible comme cela peut  être le cas pour une régression ou une classification;
    celle-ci n'est d'ailleurs pas définie dans le générateur: nous sommes dans un cas de  _unsupervised_ ou _self_supervised learning_ ?
    ''')

  def model_scatter(self):
    df = tb.get_data(self.root_path+'final_encoded_machine_dataset_noPCA.csv',index=0)
    df_train = df[df.set=='train']
    #StandardScaler
    df_train.iloc[:,5:] = (df_train.iloc[:,5:] - df_train.iloc[:,5:].mean()) / df_train.iloc[:,5:].std()
    # pca
    n_components=3
    pca = PCA(n_components=n_components)
    pca.fit(df_train.iloc[:,5:])
    df_train = df_train.join(pd.DataFrame(pca.transform(df_train.iloc[:,5:]), columns=['PCA%i' % i for i in range(n_components)], index=df_train.index))
   
    #scaler = StandardScaler()
    #df_train_vector = scaler.fit_transform(df_train_vector)
    #df_train = df_train[['fichier','set','type','machine','anomalie']].join(df_train_vector)

    st.write('''
    L'encodage des spectrogrammes permet de créer **des clusters homogènes** sans dispersion. Les algorithmes de **classification** classiques (kNN, RandomForest, SVM) testés permettent d'obtenir des résultats de classification par 
    **type de machine ou par machine prometteurs**
    - **par type de machine: ** une précision de ** classification >98,5% en moyenne** est obtenue. Quelques confusions notamment entre ToyCar et Valve. La classification par kNN reste efficace même en réduisant l'espace à 2 composantes principales, issues des 128 dimensions de l'encodage.
    - **par machine:** une précision de classification **>94 % peut être obtenue sur le jeu de train** mais de l'ordre de 90% sur le jeu de test. Les confusions se situent toujours également autour de la machine ToyCar. Les performances de classification sans ce type de machine sont > à 97,9% avec 10 composantes, et 20 voisins.
    
    Les matrices de confusion, présentées en bas de page, permettent de comprendre facilement sur quel type de machine se situent les pertes de performances.

    ''')

    st.subheader("représentation des vecteurs de features issues de l'apprentissage")
    color_choice = st.radio("coloration des nuages:",options=("type","machine"),key="radio_color",help="les points seront colorés selon votre choix, en fonction du type de la machine, ou de la machine elle-même ")

    fig = px.scatter_3d(df_train[df_train.type.isin(self.selected_machine)], x='PCA0', y='PCA1', z='PCA2',
              color=color_choice, opacity = 0.6, width=1200, height=1000, title="représentation des 3 axes principaux de l'ACP",color_discrete_sequence=self.px_color)
  
    fig.update_layout(
        title={
            'x':0.5,
            'y':0.95,
            'xanchor': 'center',
            'font':{'size':22}})
    fig.update_traces(
        marker=dict(
        size=3,
    ))
    st.plotly_chart(fig)

    col1,col2 = st.columns(2)
    col1.subheader("classification par type de machine: matrice de confusion")
    col1.image(self.root_path+"images/confusion_classif_type.png", caption="matrice de confusion classification par type")
    col2.subheader("classification par machine: matrice de confusion")
    col2.image(self.root_path+"images/confusion_classif_machine.png",caption="matrice de confusion classification par machine")

  def model_anomaly(self):

    df = tb.get_data(self.root_path+'final_encoded_machine_dataset_noPCA.csv',index=0)
    df_train = df[(df.set=='train')& (df.type.isin(self.selected_machine))]
    #StandardScaler
    df_train.iloc[:,5:] = (df_train.iloc[:,5:] - df_train.iloc[:,5:].mean()) / df_train.iloc[:,5:].std()
    # pca
    n_components=2
    pca = PCA(n_components=n_components)
    pca.fit(df_train.iloc[:,5:])

    df = df.join(pd.DataFrame(pca.transform(df.iloc[:,5:]), columns=['PCA%i' % i for i in range(n_components)], index=df.index))
    
    st.subheader("représentation des features extraites des données de test:")
    st.write("L'encodage, entraîné uniquement sur les données du set _train_, permet d'extraire des informations très intéressantes pour discriminer les anomalies de la majorité des machines:")
    
    for m in df_train.type.unique():
      with st.expander(m):
        col1,col2= st.columns(2)
        for i,mach in enumerate(df[(df.type==m)].machine.unique()):
            fig = px.scatter(df[((df.machine == mach)) & (((df.set=='train')&(df.anomalie=='non')) | ((df.set=='test')&(df.anomalie=='oui')))], x='PCA0', y='PCA1',
                    color='anomalie', opacity = 0.6, width=700, height=400, title="machine "+mach,color_discrete_sequence=self.px_color)
            fig.update_layout(
              title={
                  'x':0.5,
                  'xanchor': 'center',
                  'font':{'size':18}},
              margin = dict(l=0, r=0, b=0)
              )
            fig.update_traces(marker=dict(size=5))

            col = i%2
            if col==1:
              with col2:
                st.plotly_chart(fig)
            else:
              with col1:
                st.plotly_chart(fig)
    st.subheader("résultats de classification:")
    st.write('''

    En analysant finement les résultats d'une **classification des machines par SVM ou par kNN** par exemple, on remarque que la différenciation entre sons normaux et anormaux est plus marquée lorsque l'on tronque 
    une très grande majorité des détails issus de la décomposition en composantes principales. Ainsi, en gardant **50% des composantes principales**, un SVM classifie bien les **sons normaux à 95%** dans le jeu de train, et **seulement à 54% dans le jeu de test pour les normaux**, et **28% pour les anormaux**.
    En conservant uniquement **2 composantes principales**, ce même SVM classifie les **sons normaux à 90.66% de précision sur les données d'apprentissage**, et généralise à ** 87% sur les données de test normales**, contre **47.2% sur les sons anormaux**

    De même, un **classifieur kNN** dont l'apprentissage a été fait uniquement sur des données normales, et testé sur les données normales et anormales montre des résultat similaires. ''')

    st.success('''
    Notre objectif semble rempli:  **les classifieurs semblent en difficulté pour reconnaître avec fiabilité les machines lorsque celles-ci présentent une anormalité**, alors que la classification sur les 
    données de test normales présentent des performances conformes au jeu de validation issu du jeu d'apprentissage.
    C'était bien le but de la démarche d'encodage.  
    
    **Nous avons donc réussi à développer un modèle encodage qui permet de construire un nouvel espace de représentions discriminant les sons normaux des sons anormaux.**
    
    Les algorithmes **de détection non supervisée d'anomalies** peuvent maintenant être étudiés pour répondre à l'objectif complet du projet.
    
    ''')
    col1, col2 = st.columns(2)
    with col1:
      col1.subheader("matrice de confusion sur données de test normales")
      col1.image(self.root_path+"images/confusion_machine_test_normale.png")
    with col2:
      col2.subheader("matrice de confusion sur données de test anormales")
      col2.image(self.root_path+"images/confusion_machine_test_anormale.png")

    
  def model_detection(self):
    st.subheader("détection non supervisée des anomalies")
    st.write('''
    La stratégie de détection des anomalies est basée sur des dérivés d'algorithmes de classification. Ces algorithmes étudient la probabilité qu'un point non connu jusqu'ici, appartienne à la même distribution qu'un ensemble de points connus.

    Pour notre projet, nous avons plus particulièrement étudié les algorithmes de **LOF: _Local Outlier Factor_** et **_One-Class SVM_**
    mais il en existe bien d'autres. 
    
    En analysant les points dans leur ensemble, on constate que **les espaces
    de sons anormaux chevauchent parfois les espaces de sons normaux d'autres machines**. 
    Nous avons donc choisi d'étudier la **détection non supervisée d'anomalie machine par machine** afin de ne pas être perturbés par ce phénomène.
    ''')
    st.info('''
    **Remarque:** Ce fonctionnement représente mieux **l'application industrielle** qui pourrait en être faite: 
    on entraîne un modèle de _LOF_ ou de _OSVM_ sur des données de machine normale
    et on cherche à détecter les anomalies sur cette même machine.''')

    st.subheader('**OcSVM: One Class SVM**')
    st.markdown('''
    Le **_One Class SVM_** est un algorithme de recherche non supervisée d'anomalies, très largement utilisé.
    La famille des _machines à vecteur supports_ permet de calculer **une frontière de décision** dans un espace multidimensionnel, permettant de **classer tout nouveau point comme appartenant à une région, ou une autre**.
    
    Dans le cas du **_OSVM_**, l'algorithme **apprend à définir une région unique** à partir des points de l'entraînement. 
    Si un nouveau point testé **n'appartient pas à cette région**, il est considéré comme **_anomalie_**.
    L'OSVM que nous avons entraîné pour chaque machine repose sur un **noyau _RBF_**. 
    
    Ses performances dépendent principalement des paramètres ** $gamma$ ** et ** $nu$ **
    ''')
    st.image(self.root_path+'images/reglage-gamma-OSVM3.png', caption="exemple de contours des régions apprises suivant les paramètres gamma  (de gauche à droite) et nu (les différentes régions dessinées) ")
    #st.image(self.root_path+'images/reglage-gamma-OSVM2.png', caption="exemple de contours des régions apprises suivant les paramètres gamma  (de gauche à droite) et nu (les différentes régions dessinées) ")
    
    st.subheader('**LOF: Local Outlier Factor**')
    st.write('''
    Le _LOF_ est un algorithme de détection d'anomalie non supervisé, qui consiste à **mesurer la différence de densité autour d'un nouveau point par rapport à ses voisins**, connus et normaux. 
    
    Un nouveau point, situé au milieu d'autres points connus lors de l'entraînement, sera supposé appartenir à ce même ensemble de points normaux, alors qu'un nouveau point apparaissant très éloigné d'un nuage de point connu sera supposé être une anomalie.
    
    Chaque **nouveau point** présenté au modèle se voit donc gratifié **d'un score** d'appartenance à l'ensemble des points normaux. 
    Un **seuillage de ce score permet de le classer comme _normal_ ou _anormal_ **. 
    Ce seuillage permettant la classification comme anomalie dépend d'un paramètre appelé **_contamination_**.     
    ''')
    col1,col2 = st.columns(2)
    with col1:
      st.image(self.root_path+'images/LOF_scoring1.PNG', caption="exemple de scores _LOF_ sur des données normales/anormales séparables ")
    with col2:
      st.image(self.root_path+'images/LOF_scoring2.PNG', caption="exemple de scores _LOF_ sur des données normales/anormales difficilement séparables")

   
    st.info('''
    Nos travaux ont consisté à tester ces algorithmes et **optimiser leurs hyperparamètres** afin d'obtenir des **résultats optimaux de détection des anomalies**. 
    Pour cela, des tests de grilles de paramètres (__GridSearchCV__) ont permis de trouver les meilleurs compromis entre **_sensibilité_** 
    (détection de vraies anomalies) et **_spécificité_** (détection de vrais signaux normaux). 

    La maximisation de l'_aire sous la courbe_ (**AUC**) permet de synthétiser ce compromis _sensibilité vs spécificité_

    ''')

  def resultats(self):
    st.header("Résultats")
    result_LOF = tb.get_data_LOF()
    result_LOF = result_LOF.loc[result_LOF.type.isin(self.selected_machine)]
    # result_LOF = result_LOF.loc[result_LOF.machine.isin(self.selected_lachines)]
    result_SVM =  tb.get_data_SVM()
    result_SVM = result_SVM.loc[result_SVM.type.isin(self.selected_machine)]

    seuil = st.sidebar.select_slider('contrôle de la sensibilité',options=range(len(result_LOF.contamination.unique())), key='seuil')
    contamination = result_LOF.contamination.unique()[seuil]
    nu = result_SVM.nu.unique()[seuil]

    gp_LOF = result_LOF.groupby('contamination').mean()
    gp_LOF['algo']='LOF'
    gp_SVM = result_SVM.groupby('nu').mean()
    gp_SVM['algo']='SVM'
    test = pd.concat([gp_LOF,gp_SVM],ignore_index=True)
     
    col1, col2, col3 = st.columns((2,4,1))

    res_SVM = result_SVM.loc[result_SVM.nu==nu]
    res_SVM_aggreg = res_SVM.groupby(['nu']).mean()

    res_LOF = result_LOF.loc[result_LOF.contamination==contamination]
    res_LOF_aggreg = res_LOF.groupby(['contamination']).mean()

    fig = px.line(test, x="sensibilite", y="specificite", color='algo',color_discrete_sequence=self.px_color)
    fig.update_layout(height=300,margin = dict(t=0, l=0, r=50, b=0), title_font_size=4)
    fig.update_traces(line=dict( width=3))
    fig.add_vline(x=res_LOF_aggreg.sensibilite.iloc[0], line_width=2, line_dash="dash", line_color=self.px_color[2])
    fig.add_hline(y=res_LOF_aggreg.specificite.iloc[0], line_width=2, line_dash="dash", line_color=self.px_color[2])
    col2.write(fig)

    col1.write(' - **LOF**  : taux de contamination de {:.2%} \n   - **sensibilité: {:.2%}**\n   - **spécificité: {:.2%}**'.format(contamination,res_LOF_aggreg.sensibilite.iloc[0],res_LOF_aggreg.specificite.iloc[0]))
    col1.write(' - **OcSVM**: nu de {:.3f} \n   - **sensibilité: {:.2%}**\n   - **spécificité: {:.2%}**'.format(nu,res_SVM_aggreg.sensibilite.iloc[0],res_SVM_aggreg.specificite.iloc[0]))
      
    col1,col2 = st.columns(2)
    with col1:
        st.subheader("détection par LOF")
       
        x = [[],[]]
        for t in result_LOF['type'].unique():
          sous_cat = result_LOF[result_LOF['type']==t].machine.unique()
          x[0].extend([t for i in range(len(sous_cat))])
          x[1].extend(sous_cat)

        fig = go.Figure(go.Bar(x=x, y=res_LOF.specificite, name='spécificité',marker={'color': self.px_color[2]}))
        fig.add_trace(go.Bar(x=x, y=res_LOF.sensibilite, name='sensibilité',marker={'color': self.px_color[0]})) #6

        fig.update_xaxes(categoryorder='category ascending')
        fig.update_layout(height=200,margin = dict(t=30, l=0, r=50, b=0), title_font_size=4)
        fig.update_layout(
                  title={
                      'x':0.5,
                      'xanchor': 'center',
                      'font':{'size':18}},)
        st.write(fig)

        df_agg_error =  res_LOF.groupby('type').agg({'sensibilite': ['mean', 'min', 'max'],'specificite':['mean','min','max']})
        df_agg_error['sensibilite']['min'] = df_agg_error['sensibilite']['mean']-df_agg_error['sensibilite']['min']
        df_agg_error['sensibilite']['max'] = df_agg_error['sensibilite']['max']-df_agg_error['sensibilite']['mean']
        df_agg_error['specificite']['min'] = df_agg_error['specificite']['mean']-df_agg_error['specificite']['min']
        df_agg_error['specificite']['max'] = df_agg_error['specificite']['max']-df_agg_error['specificite']['mean']

        fig2 = go.Figure()
        fig2.add_trace(go.Bar(
            name='specificite',
            x=df_agg_error.index, y=df_agg_error['specificite']['mean'],
            marker={'color': self.px_color[2]},
            error_y=dict(
              type='data',
              symmetric=False,
              array=df_agg_error[('specificite','max')],
              arrayminus=df_agg_error[('specificite','min')])
        ))
        fig2.add_trace(go.Bar(
            name='sensibilité',
            x=df_agg_error.index, y=df_agg_error['sensibilite']['mean'],
            marker={'color': self.px_color[0]},#6
            error_y=dict(
              type='data',
              symmetric=False,
              array=df_agg_error[('sensibilite','max')],
              arrayminus=df_agg_error[('sensibilite','min')])
        ))
        fig2.update_layout(barmode='group',height=200,margin = dict(t=30, l=0, r=50, b=0), title_font_size=4 )
        fig2.update_xaxes(categoryorder='category ascending')
        col1.write(fig2)

    with col2:
        st.subheader("détection par OcSVM")

        x = [[],[]]
        for t in result_SVM['type'].unique():
          sous_cat = result_SVM[result_SVM['type']==t].machine.unique()
          x[0].extend([t for i in range(len(sous_cat))])
          x[1].extend(sous_cat)

        
        fig = go.Figure(go.Bar(x=x, y=res_SVM.specificite, name='spécificité',marker={'color': self.px_color[2]}))
        fig.add_trace(go.Bar(x=x, y=res_SVM.sensibilite, name='sensibilité',marker={'color': self.px_color[1]})) #6

        fig.update_xaxes(categoryorder='category ascending')
        fig.update_layout(height=200,margin = dict(t=30, l=0, r=50, b=0), title_font_size=4)
        fig.update_layout(
                  title={
                      'x':0.5,
                      'xanchor': 'center',
                      'font':{'size':18}},)
        st.write(fig)

        df_agg_error =  res_SVM.groupby('type').agg({'sensibilite': ['mean', 'min', 'max'],'specificite':['mean','min','max']})
        df_agg_error['sensibilite']['min'] = df_agg_error['sensibilite']['mean']-df_agg_error['sensibilite']['min']
        df_agg_error['sensibilite']['max'] = df_agg_error['sensibilite']['max']-df_agg_error['sensibilite']['mean']
        df_agg_error['specificite']['min'] = df_agg_error['specificite']['mean']-df_agg_error['specificite']['min']
        df_agg_error['specificite']['max'] = df_agg_error['specificite']['max']-df_agg_error['specificite']['mean']

        fig2 = go.Figure()
        fig2.add_trace(go.Bar(
            name='specificite',
            x=df_agg_error.index, y=df_agg_error['specificite']['mean'],
            marker={'color': self.px_color[2]}, 
            error_y=dict(
              type='data',
              symmetric=False,
              array=df_agg_error[('specificite','max')],
              arrayminus=df_agg_error[('specificite','min')])
        ))
        fig2.add_trace(go.Bar(
            name='sensibilité',
            x=df_agg_error.index, y=df_agg_error['sensibilite']['mean'],
            marker={'color': self.px_color[1]}, #6
            error_y=dict(
              type='data',
              symmetric=False,
              array=df_agg_error[('sensibilite','max')],
              arrayminus=df_agg_error[('sensibilite','min')])
        ))
        fig2.update_layout(barmode='group',height=200,margin = dict(t=30, l=0, r=50, b=0), title_font_size=4 )
        fig2.update_xaxes(categoryorder='category ascending')
        col2.write(fig2)

    st.header("Perspectives")
    st.subheader("Détection d'anomalies")
    st.write('''
    - meilleure **utilisation des ressources matérielles** (TPU pour créer les spectrogrammes en mois de 1,5 ms et GPU pour entraînement)
    - réseau d'encodage
      - optimiser le **réseau d'encodage**: nombre et taille des couches, dimensions de sorties, résolution du spectrogramme
      - utiliser les **RNN** pour encoder 
      - optimiser la **structuration des données** pour l'apprentissage du embedding / optimiser la **mémoire** pour l'apprentissage (25 Go nécessaires aujourd'hui)
      - optimiser et **approfondir l'apprentissage spécifiquement** autour de quelques machines ?
       balance des classes, random undersampling/oversampling car la spécificité semble liée à la distribution des machines dans le set de train (valve02, ? 
    - détection d'anomalies
      - utilisation de la détection en **streaming**, sur des périodes plus courtes 
      - aller plus loin dans le choix et l'optimisation d'**algorithmes de détection d'anomalies** 
    ''')
    st.subheader("Application réelle")
    st.write('''
    - transposition du modèle sur d'autres sons issus d'**autres environnements**
    - **exploitation dans le cadre d'une supervision d'installations**
    ''')
    #

  def update_samples(self):
    st.session_state.df_samples = np.random.choice(len(self.df),20,False)
    for m in self.selected_machine:
      st.session_state['sample_normal_'+m]  = self.df[(self.df['type']==m) & (self.df["anomalie"]=='non')].sample(1).fichier.values[0]
      st.session_state['sample_anormal_'+m]  = self.df[(self.df['type']==m) & (self.df["anomalie"]=='oui')].sample(1).fichier.values[0]

  def a_propos(self):
    st.markdown("""
    # A propos
    **pyGoodVibes est un travail collaboratif** réalisé dans le cadre du parcours de formation [DataScientist](https://datascientest.com/formation-data-scientist) avec l’organisme *DataScienTest* et *l’Université Paris La Sorbonne*.
    
    L’application Streamlit a été conçue à des fins de **présentation du projet de fin d’étude**, sur un jeu de données issu du challenge [DCASE2020](http://dcase.community/challenge2020/task-unsupervised-detection-of-anomalous-sounds), disponible librement sur Kaggle.

    Pour toute besoin d’information ou d’utilisation des supports et des résultats de pyGoodVibes, merci de faire parvenir vos demandes directement aux auteurs.
    """)

    col1, col2, col3, = st.columns(3)
    with col1:
      st.image(f"{self.root_path}images/vignette aurelien.png")
      st.image(f"{self.root_path}images/vignette christophe.png")
    with col2:
      st.write("")
      st.write("")
      st.markdown("""
      **Aurélien NANETTE**

      _Data Scientist chez LISEA Concessionnaire de la LGV SEA_

      [aurelien.nanette@gmail.com](aurelien.nanette@gmail.com)

      [https://www.linkedin.com/in/a-nanette/](https://www.linkedin.com/in/a-nanette/)

      """)
      st.write("")
      st.write("")
      st.write("")
      st.write("")


      

      st.markdown("""
      **Christophe GAUFFRE**
      
      _Consultant Freelance Vision & IT pour l'industrie_

      [contact@c-gauffre.fr](contact@c-gauffre.fr)

      [https://www.linkedin.com/in/christophegauffre/](https://www.linkedin.com/in/christophegauffre/)


      """)


  def biblio(self):
    st.write('''
    - [What’s wrong with CNNs and spectrograms for audio processing?](https://towardsdatascience.com/whats-wrong-with-spectrograms-and-cnns-for-audio-processing-311377d7ccd)
    - [How to use machine learning for anomaly detection and condition monitoring](https://towardsdatascience.com/how-to-use-machine-learning-for-anomaly-detection-and-condition-monitoring-6742f82900d7)
    - [5 Ways to Detect Outliers/Anomalies](https://towardsdatascience.com/5-ways-to-detect-outliers-that-every-data-scientist-should-know-python-code-70a54335a623)
    - [embeddings et représentations continues](https://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526)
    - [outlier et distance de mahalanobis](https://towardsdatascience.com/multivariate-outlier-detection-in-python-e946cfc843b3)
    - [Useful methods in Python for Detecting Outliers in Data](https://towardsdatascience.com/mastering-outlier-detection-in-python-61d1090a5b08)
    - [Unsupervised Anomaly Detection for Univariate & Multivariate Data](https://towardsdatascience.com/anomaly-detection-for-dummies-15f148e559c1)
    - [Anomaly Detection, A Key Task for AI and Machine Learning, Explained](https://www.kdnuggets.com/2019/10/anomaly-detection-explained.html)
    - [Unsupervised Anomaly Detection in Flight Data Using Convolutional Variational Auto-Encoder](https://www.mdpi.com/2226-4310/7/8/115)
    - [knn for anomalies detection](https://towardsdatascience.com/k-nearest-neighbors-knn-for-anomaly-detection-fdf8ee160d13)
    - à implémenter tellement c'est simple:[hands-on-unsupervised-learning](https://www.oreilly.com/library/view/hands-on-unsupervised-learning/9781492035633/ch04.html)
    - transformer et vecteur d'attention (en complément des RNN): ??
    - [BERT models et dérivés (ALBERT,AALBERT) basés sur les Transformer Models](https://medium.com/codex/self-supervised-deep-learning-on-audio-4daf2f2c51f5)
    - [self supervised approch, contrastive learning](https://generallyintelligent.ai/understanding-self-supervised-contrastive-learning.html) utilisation de batch normalisation nécessaire!
    - [A Comparative Evaluation of Unsupervised Anomaly Detection Algorithms for Multivariate Data](https://www.researchgate.net/publication/301533547_A_Comparative_Evaluation_of_Unsupervised_Anomaly_Detection_Algorithms_for_Multivariate_Data)
    - [Automatic Hyperparameter Tuning Method for Local Outlier Factor, with Applications to Anomaly Detection](https://ieeexplore.ieee.org/abstract/document/9006151)
    ''')

  def plot_audio(self, file,fig,ax):

    audio_data,sr=librosa.load(file)
    dt = 1 / sr
    t = dt * np.arange(len(audio_data))

    librosa.display.waveshow(audio_data, sr=sr, ax=ax)
    ax.set_ylabel('amplitude')
    ax.set_xlabel('temps (s)')
    ax.label_outer()

    return fig
      
  def plot_spectrogram(self, audio_file, ax):
    """
    calcul et affiche le spectrogramme d'un fichier audio
    :param audio:signal audio à transformer 
    :param fe: fréquence d'échantillonnage de l'audio utilisé
    :param dt: int>0: résolution temporelle de la STFT
    :return: None
    """
    audio_data,sr=librosa.load(audio_file)
    dt = 1 / sr
    t = dt * np.arange(len(audio_data))

    n_fft = 1024
    hop_length = int(n_fft/4)
    D_highres = librosa.stft(audio_data, n_fft = n_fft, hop_length=hop_length)
    S_db = librosa.amplitude_to_db(np.abs(D_highres), ref=np.max)
    img = librosa.display.specshow(S_db, x_axis='time', hop_length=hop_length,
                                  y_axis='hz' ,ax=ax, cmap='jet')
    ax.set_ylabel('Frequence (Hz)')
    ax.set_xlabel('temps (s)')
    ax.label_outer()

    
    return img,S_db

def main():

  st.set_option('deprecation.showPyplotGlobalUse', False)
  apptitle = 'pyGoodVibes'
  st.set_page_config(page_title=apptitle, page_icon=":construction_worker:",layout = "wide")
  app = my_app()
    
if __name__ == '__main__':
  main()

Overwriting app.py


## tunneling ngrok-streamlit

### Lancement du tunnel entre ngrok et l'application streamlit

In [37]:
from pyngrok import ngrok

!ngrok authtoken 1xYQesOXezZpo4IBed4ePF72elz_6J7QhKowaT1Nuaq2pXBVT  # Christophe

# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(8501)

!streamlit run app.py &>/dev/null& # &>/dev/null& rend non bloquant. 

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# Lancement du tunnel

In [38]:
print("RDV à cette adresse:")
print(public_url)

RDV à cette adresse:
NgrokTunnel: "http://4465-34-82-44-143.ngrok.io" -> "http://localhost:8501"


# Terminer le tunnel de Ngrok

In [36]:
!pgrep streamlit
ngrok.kill()

481
1039
1143


# Bibliographie

**sources intéressantes**
- représentations continues de Paul Dechorgnat
- [What’s wrong with CNNs and spectrograms for audio processing?](https://towardsdatascience.com/whats-wrong-with-spectrograms-and-cnns-for-audio-processing-311377d7ccd)
- [How to use machine learning for anomaly detection and condition monitoring](https://towardsdatascience.com/how-to-use-machine-learning-for-anomaly-detection-and-condition-monitoring-6742f82900d7)
- [5 Ways to Detect Outliers/Anomalies](https://towardsdatascience.com/5-ways-to-detect-outliers-that-every-data-scientist-should-know-python-code-70a54335a623)
- [embeddings et représentations continues](https://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526)
- [outlier et distance de mahalanobis](https://towardsdatascience.com/multivariate-outlier-detection-in-python-e946cfc843b3)
- [Useful methods in Python for Detecting Outliers in Data](https://towardsdatascience.com/mastering-outlier-detection-in-python-61d1090a5b08)
-[Unsupervised Anomaly Detection for Univariate & Multivariate Data](https://towardsdatascience.com/anomaly-detection-for-dummies-15f148e559c1)
- [Anomaly Detection, A Key Task for AI and Machine Learning, Explained](https://www.kdnuggets.com/2019/10/anomaly-detection-explained.html)
- [Unsupervised Anomaly Detection in Flight Data Using Convolutional Variational Auto-Encoder](https://www.mdpi.com/2226-4310/7/8/115)
- [knn for anomalies detection](https://towardsdatascience.com/k-nearest-neighbors-knn-for-anomaly-detection-fdf8ee160d13)
- à implémenter tellement c'est simple:[hands-on-unsupervised-learning](https://www.oreilly.com/library/view/hands-on-unsupervised-learning/9781492035633/ch04.html)
- transformer et vecteur d'attention (en complément des RNN)
- BERT models et dérivés (ALBERT,AALBERT) basés sur les Transformer Models
https://medium.com/codex/self-supervised-deep-learning-on-audio-4daf2f2c51f5
- [self supervised approch, contrastive learning](https://generallyintelligent.ai/understanding-self-supervised-contrastive-learning.html) utilisation de batch normalisation nécessaire!
- [A Comparative Evaluation of Unsupervised Anomaly Detection Algorithms for Multivariate Data](https://www.researchgate.net/publication/301533547_A_Comparative_Evaluation_of_Unsupervised_Anomaly_Detection_Algorithms_for_Multivariate_Data)

